In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/My\ paper

/content/drive/Shareddrives/My paper


In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime

In [ ]:
member = pd.read_csv('dataset/91APP_MemberData.csv')

In [ ]:
order = pd.read_csv('dataset/91APP_OrderData.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print(len(member['MemberID'].unique()), len(order['MemberID'].unique()))

377133 235493


In [ ]:
order['TradesDateTime'] = pd.to_datetime(order['TradesDateTime'])
order = order.sort_values(by='TradesDateTime')

In [ ]:
member['RegisterDateTime'] = pd.to_datetime(member['RegisterDateTime'])
member = member.sort_values(by='RegisterDateTime')
member = member.drop_duplicates(subset=['MemberID'], keep=False)

In [ ]:
len(member)

377132

In [ ]:
label_data = pd.read_csv('./Customer Value Prediction/TABLES/Label Data.csv')

In [ ]:
memberIDs = list(label_data['MemberID'].unique())

## 加上 UID 欄位


In [ ]:
def add_uid_col(data):
  df = data.copy()
  df['uid'] = df['MemberID'].str.replace('+', '%2B')
  df['uid'] = df['uid'].str.replace('/', '%2F')
  df['uid'] = df['uid'].str.replace('=', '%3D')

  return df['uid']

In [ ]:
label_data['uid'] = add_uid_col(label_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
uids = list(label_data['uid'].unique())

In [ ]:
print(len(memberIDs), len(uids))

83931 83931


## 讀取所有行為資料

In [ ]:
behaviors = pd.read_csv('all behaviors data.csv')

In [ ]:
behaviors.columns

Index(['Unnamed: 0', 'dc', 'did', 'uid', 'DateTime', 'bh', 'uid_filled'], dtype='object')

In [ ]:
behaviors = behaviors[behaviors['uid_filled'].isin(uids)][['did', 'uid', 'DateTime', 'bh', 'uid_filled']]

In [ ]:
len(behaviors)

12879698

In [ ]:
behaviors = behaviors.sort_values(by=['uid_filled', 'DateTime'])

In [ ]:
## session 切換三種狀況:
### 1. 隔天
### 2. 不同 traffic source
### 3. 閒置 30 min
def split_session(df):
  df_copy = df.copy()
  df_copy['Session'] = [0] * len(df)

  df_copy['DateTime'] = pd.to_datetime(df_copy['DateTime'], format='%Y-%m-%d %H:%M:%S')
  df_copy['datetime_previous'] = df_copy['DateTime'].shift(periods=1, fill_value=df_copy.iloc[0]['DateTime'])
  df_copy['datetime_next'] = df_copy['DateTime'].shift(periods=-1, fill_value='2020-04-30 23:59:59.999')

  df_copy['uid_pre'] = df_copy['uid_filled'].shift(periods=1, fill_value='first_member')
  df_copy['uid_next'] = df_copy['uid_filled'].shift(periods=-1, fill_value='final_member')
  df_copy.loc[(df_copy['uid_filled'] != df_copy['uid_next']), 'datetime_next'] = '2020-04-30 23:59:59.999'

  df_copy['TimeDiff'] = (df_copy['datetime_next'] - df_copy['DateTime']).astype('timedelta64[m]')
  df_copy.loc[(df_copy['uid_filled'] != df_copy['uid_next']), 'TimeDiff'] = 0
  df_copy['DayDiff'] = df_copy['DateTime'].dt.day - df_copy['datetime_previous'].dt.day
  df_copy['MinuteDiff'] = (df_copy['DateTime'] - df_copy['datetime_previous']).astype('timedelta64[m]')

  df_copy.loc[(
      (df_copy['bh'] == 'traffic') | 
      (df_copy['MinuteDiff']>=30) | 
      (df_copy['DayDiff'] != 0) | 
      (df_copy['uid_filled'] != df_copy['uid_pre'])
      ), 'Session'] = 1

  df_copy.loc[(df_copy['uid_filled'] != df_copy['uid_pre']), 'MinuteDiff'] = 0

  return df_copy

In [ ]:
split_session_data = split_session(behaviors).reset_index()
split_session_data['SessionOrder'] = split_session_data['Session'].cumsum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Passing <class 'str'> to shift is deprecated and will raise in a future version, pass Timestamp instead.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
len(split_session_data['uid_filled'].unique())

83931

In [ ]:
pd.DataFrame(split_session_data.groupby(by=['SessionOrder','bh']).size().unstack()).fillna(0).astype(int)

bh,add,checkout,purchase,register,search,traffic,viewactivity,viewcategory,viewmainpage,viewproduct
SessionOrder,,,,,,,,,,
1,0,0,0,0,0,0,1,0,6,3
2,2,0,0,0,0,0,0,6,17,6
3,0,0,0,0,0,0,0,0,6,1
4,1,0,0,1,0,1,0,7,3,12
5,2,9,0,0,0,1,4,10,19,13
...,...,...,...,...,...,...,...,...,...,...
4610620,0,0,0,0,0,0,0,0,2,1
4610621,0,0,0,0,1,0,0,2,2,1
4610622,2,0,0,0,0,0,0,11,2,11


In [ ]:
sessions = list(split_session_data.drop_duplicates(subset=['SessionOrder'], keep='first').groupby(by=['uid_filled']).tail(12)['SessionOrder'].unique())
sessions_df = split_session_data[split_session_data['SessionOrder'].isin(sessions)]

bh_df = pd.DataFrame(sessions_df.groupby(by=['SessionOrder','bh']).size().unstack()).fillna(0).astype(int)

In [ ]:
session_group = sessions_df.groupby(by=['SessionOrder'])

bh_df['TimeDiff'] = session_group.first()['MinuteDiff']
bh_df['TotalTime'] = pd.DataFrame(session_group.sum())['MinuteDiff'] - bh_df['TimeDiff']
bh_df = bh_df.fillna(0).astype(int)

result_df = pd.merge(bh_df, session_group.tail(1)[['SessionOrder', 'uid_filled']], on='SessionOrder', how='left')

In [ ]:
result_df

,SessionOrder,add,checkout,purchase,register,search,traffic,viewactivity,viewcategory,viewmainpage,viewproduct,TimeDiff,TotalTime,uid_filled
0,74,0,0,0,0,0,1,0,0,0,0,0,0,%2B%2B%2FsALKlNdkh585nb%2F9geIrN84FIZZLkD8qf90...
1,75,0,0,0,0,0,1,0,0,0,0,0,0,%2B%2B%2FsALKlNdkh585nb%2F9geIrN84FIZZLkD8qf90...
2,76,0,0,0,0,0,1,0,0,0,0,0,0,%2B%2B%2FsALKlNdkh585nb%2F9geIrN84FIZZLkD8qf90...
3,77,0,0,0,0,0,1,0,0,0,0,0,0,%2B%2B%2FsALKlNdkh585nb%2F9geIrN84FIZZLkD8qf90...
4,78,0,0,0,0,0,0,0,0,4,1,49124,1,%2B%2B%2FsALKlNdkh585nb%2F9geIrN84FIZZLkD8qf90...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700508,4610620,0,0,0,0,0,0,0,0,2,1,26912,0,zzwvJvTcevm90Q6BkxJqud94QGdwcvDOrWT88F1cTM4%3D
700509,4610621,0,0,0,0,1,0,0,2,2,1,49404,0,zzwvJvTcevm90Q6BkxJqud94QGdwcvDOrWT88F1cTM4%3D
700510,4610622,2,0,0,0,0,0,0,11,2,11,1220,3,zzwvJvTcevm90Q6BkxJqud94QGdwcvDOrWT88F1cTM4%3D
700511,4610623,0,0,0,0,1,0,0,0,1,4,26979,1,zzwvJvTcevm90Q6BkxJqud94QGdwcvDOrWT88F1cTM4%3D


In [ ]:
len(result_df['uid_filled'].unique())

83931

In [ ]:
result = pd.merge(label_data[['MemberID', 'uid']], result_df, left_on='uid', right_on='uid_filled', how='left').drop(['uid'], axis=1)

In [ ]:
len(result['uid_filled'].unique())

83931

In [ ]:
len(result)

700513

In [ ]:
result = pd.merge(result, label_data[['MemberID', 'TotalSalesAmount', 'Active', 'VIP']], on='MemberID', how='left').fillna(0)

In [ ]:
len(result['uid_filled'].unique())

83931

In [ ]:
result.head()

,MemberID,SessionOrder,add,checkout,purchase,register,search,traffic,viewactivity,viewcategory,viewmainpage,viewproduct,TimeDiff,TotalTime,uid_filled,TotalSalesAmount,Active,VIP
0,RSQpIPxqSK2ooLPuUwWw8HhTN73mFGsmGPEmdarbdGM=,2127581,1,2,1,0,0,0,0,9,15,4,0,22,RSQpIPxqSK2ooLPuUwWw8HhTN73mFGsmGPEmdarbdGM%3D,0.0,0.0,0.0
1,AqBrlkHltks4neA3Fmtq+yfiI4F7aZGODqKPeF3zGsA=,921405,0,0,0,0,0,0,0,0,1,0,0,0,AqBrlkHltks4neA3Fmtq%2ByfiI4F7aZGODqKPeF3zGsA%3D,0.0,0.0,0.0
2,AqBrlkHltks4neA3Fmtq+yfiI4F7aZGODqKPeF3zGsA=,921406,3,2,2,0,0,0,0,5,6,22,17443,7,AqBrlkHltks4neA3Fmtq%2ByfiI4F7aZGODqKPeF3zGsA%3D,0.0,0.0,0.0
3,AqBrlkHltks4neA3Fmtq+yfiI4F7aZGODqKPeF3zGsA=,921407,0,0,0,0,0,0,0,0,1,0,3896,0,AqBrlkHltks4neA3Fmtq%2ByfiI4F7aZGODqKPeF3zGsA%3D,0.0,0.0,0.0
4,AqBrlkHltks4neA3Fmtq+yfiI4F7aZGODqKPeF3zGsA=,921408,1,1,1,0,0,0,0,2,4,9,1585,30,AqBrlkHltks4neA3Fmtq%2ByfiI4F7aZGODqKPeF3zGsA%3D,0.0,0.0,0.0


In [ ]:
result.isna().sum()

MemberID            0
SessionOrder        0
add                 0
checkout            0
purchase            0
register            0
search              0
traffic             0
viewactivity        0
viewcategory        0
viewmainpage        0
viewproduct         0
TimeDiff            0
TotalTime           0
uid_filled          0
TotalSalesAmount    0
Active              0
VIP                 0
dtype: int64

In [ ]:
result.to_csv('Session Behaviors Count.csv', index=False)

In [ ]:
len(result)

700513